In [1]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [3]:
%%time
infile = open('./data/500_QMC_dgp.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.8  0.1  1.2]']
print(Y_dgp.shape)

(2798, 500)
Wall time: 14.4 ms


In [4]:
%%time
print(Y_dgp[:,0])
result = []
draws = 8
start = 0
end = 50
method = 'BQMC'
for i in range(start,end, 10):
    for j in range(i,i+10):
        print('iteration: %i' %(j+1))
        qmc = MMNL(X,Y_dgp[:,j],draws,3,method=method)
        solution = qmc.solver()
        result.append(solution)
    pickle.dump( result, open( "./resultaten/%s(%i)/%i-%i_%s(%i)_results.p" %(method,draws,i,i+10,method,draws), "wb" ) )
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

[1. 1. 1. ... 3. 3. 3.]
iteration: 1


C:\Users\niels\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:502: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  RuntimeWarning)


ValueError: operands could not be broadcast together with shapes (3,8) (0,1) 

In [10]:
x_est = np.array(result)
x_est.shape

(100, 6)

In [11]:
np.mean(x_est, axis=0)

array([ 4.58777441,  2.65487859, -7.12572568,  7.45172249,  2.22983804,
        5.23629935])